In [1]:
from decouple import config, AutoConfig

In [2]:
config = AutoConfig(search_path='.')

In [3]:
assert config("ALPHA_VANTAGE_API_KEY", default=None, cast=str) is not None

In [4]:
ALPHA_VANTAGE_API_KEY = config("ALPHA_VANTAGE_API_KEY", default=None, cast=str)

In [5]:
ALPHA_VANTAGE_API_KEY

'ANIJLNK3PVWBJLR6'

In [6]:
# import requests

# params = {
#     "api_key" : ALPHA_VANTAGE_API_KEY,
#     "ticker": "AAPL",
#     "function": "TIME_SERIES_INTRADAY",
#     "interval": "5min",
# }

# # replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
# url = 'https://www.alphavantage.co/query?function={function}&symbol={ticker}&interval={interval}&apikey={api_key}'.format(
#     **params
# )

# r = requests.get(url)
# data = r.json()

In [7]:
# dataset_key = [x for x in list(data.keys()) if not x == "Meta Data"][0]

In [8]:
# results = data[dataset_key]
# print(results)

In [9]:
# timestamp_str = list(results.keys())[0]
# timestamp_str

In [10]:
# result_val = results.get(timestamp_str)

In [11]:
# import pytz
# from datetime import datetime
# from decimal import Decimal

# def transform_alpha_vantage_result(timestamp_str, result):
#     timestamp_format = "%Y-%m-%d %H:%M:%S"
#     eastern = pytz.timezone("US/Eastern")
#     utc = pytz.utc

#     utc_timestamp = eastern.localize(datetime.strptime(timestamp_str, timestamp_format)).astimezone(utc)

#     #unix_timestamp = result.get('t') / 1000.0
#     #utc_timestamp = datetime.fromtimestamp(unix_timestamp, tz=pytz.timezone('UTC'))

    
#     return {
#         'open_price': Decimal(result['1. open']),
#         'close_price': Decimal(result['4. close']),
#         'high_price': Decimal(result['2. high']),
#         'low_price': Decimal(result['3. low']),
#         'number_of_trades': None,
#         'volume': int(result['5. volume']),
#         'volume_weighted_average': None,
#         'time': utc_timestamp,
#     }

In [12]:
# transform_alpha_vantage_result(timestamp_str, result_val)

In [13]:
# for timestamp_str in results.keys():
#    new_data = transform_alpha_vantage_result(timestamp_str, results.get(timestamp_str))
#    print(new_data)

In [14]:
import pytz
from typing import Literal
from datetime import datetime
from dataclasses import dataclass
from decimal import Decimal
from urllib.parse import urlencode
import requests

ALPHA_VANTAGE_API_KEY = config("ALPHA_VANTAGE_API_KEY", default=None, cast=str)

def transform_alpha_vantage_result(timestamp_str, result):
    timestamp_format = "%Y-%m-%d %H:%M:%S"
    eastern = pytz.timezone("US/Eastern")
    utc = pytz.utc

    utc_timestamp = eastern.localize(datetime.strptime(timestamp_str, timestamp_format)).astimezone(utc)
    
    return {
        'open_price': Decimal(result['1. open']),
        'close_price': Decimal(result['4. close']),
        'high_price': Decimal(result['2. high']),
        'low_price': Decimal(result['3. low']),
        'number_of_trades': None,
        'volume': int(result['5. volume']),
        'volume_weighted_average': None,
        'time': utc_timestamp,
    }

@dataclass
class AlphaVantageAPIClient:
    ticker:str = "AAPL"
    function: Literal["TIME_SERIES_INTRADAY"] = "TIME_SERIES_INTRADAY"
    interval: Literal["1min", "5min", "15min", "30min", "60min"] = "5min"
    month:str = "2024-01"
    api_key:str = ""

    def get_api_key(self):
        return self.api_key or ALPHA_VANTAGE_API_KEY
            
    def get_params(self):
        return {
            "apikey": self.get_api_key(),
            "symbol": self.ticker,
            "function": self.function,
            "interval": self.interval,
            "month": self.month,
        }
    
    def get_headers(self):
        api_key = self.get_api_key()
        return {}

    def generate_url(self, pass_auth=False):
        path = '/query'
        url = f'https://www.alphavantage.co{path}'
        params = self.get_params()
        encoded_params = urlencode(params)
        url = f"{url}?{encoded_params}"
        if pass_auth:
            api_key = self.get_api_key()
            url += f"&api_key={api_key}"
        return url

    def fetch_data(self):
        headers = self.get_headers()
        url = self.generate_url()
        response = requests.get(url, headers=headers)
        response.raise_for_status() #not 200/201 status response
        return response.json()

    def get_stock_data(self):
        data = self.fetch_data()
        dataset_key = [x for x in list(data.keys()) if not x.lower() == "meta data"][0]
        results = data[dataset_key]
        dataset = []
        for timestamp_str in results.keys():
            dataset.append(
                transform_alpha_vantage_result(timestamp_str, results.get(timestamp_str))
            )
        return dataset

In [15]:
AlphaVantageAPIClient().get_stock_data()

[{'open_price': Decimal('184.5862'),
  'close_price': Decimal('184.7753'),
  'high_price': Decimal('184.8648'),
  'low_price': Decimal('184.5763'),
  'number_of_trades': None,
  'volume': 7522,
  'volume_weighted_average': None,
  'time': datetime.datetime(2024, 2, 1, 0, 55, tzinfo=<UTC>)},
 {'open_price': Decimal('184.5962'),
  'close_price': Decimal('184.5962'),
  'high_price': Decimal('184.6260'),
  'low_price': Decimal('184.4966'),
  'number_of_trades': None,
  'volume': 3030,
  'volume_weighted_average': None,
  'time': datetime.datetime(2024, 2, 1, 0, 50, tzinfo=<UTC>)},
 {'open_price': Decimal('184.6559'),
  'close_price': Decimal('184.5962'),
  'high_price': Decimal('184.6559'),
  'low_price': Decimal('184.5962'),
  'number_of_trades': None,
  'volume': 3449,
  'volume_weighted_average': None,
  'time': datetime.datetime(2024, 2, 1, 0, 45, tzinfo=<UTC>)},
 {'open_price': Decimal('184.5862'),
  'close_price': Decimal('184.6559'),
  'high_price': Decimal('184.6758'),
  'low_price